In [106]:
import sys, os
sys.path.insert(0, '/home/hektor/scripts')
#sys.path.insert(0, '/lunarc/nobackup/projects/qim')

import qim.registration as registration
import qim.tools as tools
import tifffile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from skimage import filters
import scipy.optimize as opt


## Fabricate some data

In [117]:
xedges = np.linspace(0,256,256)
yedges = xedges
xcen = tools.find_bin_centers(xedges)
ycen = tools.find_bin_centers(yedges)
Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
xy = np.vstack((X.ravel(),Y.ravel())) #to comply with curve_fit syntax

data = tools.gaussian2D(xy,64,64,20,10,np.pi/4,10,0)+tools.gaussian2D(xy,125,200,10,20,np.pi/8,15,0)+tools.gaussian2D(xy,200,100,30,10,np.pi,15,0)
data += tools.gaussian2D(xy,125,150,20,40,np.pi/2,40,0)
data=data.reshape(X.shape)
print(X.shape,data.shape)
plt.figure()
plt.pcolormesh(X,Y,data)

(255, 255) (255, 255)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [120]:
qimfolder = '/lunarc/nobackup/projects/qim/QIM_2020_LU_Elin_Bio'
datapath = os.path.join(qimfolder,'pvt21/reg-common-peaks_new')
x_data = os.path.join(datapath,'x_registered.tiff')
n_data = os.path.join(datapath,'n-bin1.tif')

x_volume = tifffile.imread(x_data)
n_volume = tifffile.imread(n_data)


In [121]:
nbins = 256
data,xedges,yedges = registration.make_dual_histogram(x_volume,n_volume,nbins)

registration.plot_dual_histogram(data,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=2500)

NameError: name 'H' is not defined

In [123]:
registration.plot_dual_histogram(data,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=2500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [131]:
#should go in dualHistogramSegmentation.py
def fit_gaussians_test(H,ml,xedges,yedges,fitdist=20,mindist=10):
    """ 
    Fit 2D gaussians to the dual histogram.
    Input:
        H: dual histogram
        ml: marker locations
        xedges: centers of bins on x-axis
        yedges: centers of bins on y-axis
    Keyword arguments:
        fitdist: Radius of circle to fit data within, default = 20
        mindist: Smallest allowed distance between two Gaussians, default = 10
    Output:
        opts: list of fitted parameters [xc,yc,sigma_x,sigma_y,theta,amplitude,offset]
    """
    xcen = tools.find_bin_centers(xedges)
    ycen = tools.find_bin_centers(yedges)

    Y,X = np.meshgrid(ycen,xcen,indexing='ij') #coordinates in dual histogram
    xy = np.vstack((X.ravel(),Y.ravel())) #to comply with curve_fit syntax
    opts = []
    #Sort the markers to always fit the highest first
    hv = [H[m[1],m[0]] for m in ml]
    idx = np.argsort(hv)
    ml = [ml[i] for i in idx]
    
    for m in reversed(ml):
        x0,y0 =xcen[m[0]],ycen[m[1]] 
        print('Trying to fit peak at ({:.1f}, {:.1f})'.format(x0,y0))
        Hc=H.copy()
        #subtract already fitted
        if len(opts)>0:
            for op in opts:
                g = tools.gaussian2D(xy,*op)
                Hc -= g.reshape(Hc.shape)
        for i in range(Hc.shape[0]):
            for j in range(Hc.shape[1]):
                d = np.sqrt((i-m[1])**2+(j-m[0])**2)
                if d > fitdist:
                    Hc[i,j] = 0
        #check if marker is on an edge
        if (m[0]==0) or (m[0]==H.shape[0]-1) or (m[1]==0) or (m[1]==H.shape[0]-1):
            print('Marker is on an edge')
            Hc = np.zeros(H.shape)
            if (m[0]==0) or (m[0]==H.shape[0]-1): #left/right edges
                Hc[:,m[0]] = H[:,m[0]]
            elif (m[1]==0) or (m[1]==H.shape[0]-1): #bottom/top edges
                Hc[m[1],:] = H[m[1],:]
                
        # Find the amplitude and position of highest value
        idx = np.argmax(Hc)
        y0i,x0i = np.unravel_index(idx,Hc.shape)
        x0 = X[y0i,x0i]
        y0 = Y[y0i,x0i]
        H0=Hc[y0i,x0i]
        of = 0 #offset
        inits = [100,100,0]
        
        #init = [x0,y0,100,100,0,H[m[1],m[0]],0]
        #print('Initial guess: [{:.3f} {:.3f} {:.1f} {:.1f} {:.1f} {:1f} {:.1f}]'.format(*init))
        try:
            #popt,pcov = opt.curve_fit(tools.gaussian2D,xy,Hc.ravel(),p0=init)
            pt,pcov = opt.curve_fit(lambda xy,sx,sy,th: tools.gaussian2D(xy,x0,y0,sx,sy,th,H0,of),xy,Hc.ravel(),p0=inits)
            popt=[x0,y0,pt[0],pt[1],pt[2],H0,of]
            print('Fitted parameters [{:.3f} {:.3f} {:.3f} {:.1f} {:.1f} {:1f} {:.1f}]'.format(*popt))
            # sanity check
            tr = np.trace(pcov)
            det = np.linalg.det(pcov)
            if (tr<=0) and (det<=0):
                print('This is not fitted to a maximum, determinant: {:.3f}, trace: {:.3f}. Skipping.'.format(tr,det))
                continue
            #Check if it is close to something already fitted
            if len(opts)>0:
                dist = [np.sqrt((popt[0]-op[0])**2+(popt[1]-op[1])**2) for op in opts]
                if np.min(dist)>mindist:
                    opts.append(popt)
                    print('Fitted {:d} peaks'.format(len(opts)))
                else:
                    print('This Gaussian is too close to something already fitted, d={:.3f}. Skipping.'.format(np.min(dist)))
            else:
                opts.append(popt)
                print('Fitted {:d} peaks'.format(len(opts)))
        except RuntimeError:
            print('Failed to fit this peak. Moving to the next.')
            continue
    return opts

In [154]:
ml = [[64,64],[125,200],[200,100],[125,150]]
ml = [[7175, 3139], [20800, 46000], [25800, 15900], [48000, 28000], [65085, 16200]]#x reg onto n
markers = registration.setup_markers(data,xedges,yedges,markers=ml)
print(markers)
opts = fit_gaussians_test(data,markers,xedges,yedges,fitdist=20,mindist=10)
registration.plot_dual_histogram(data,xedges,yedges,markers=markers,fits=opts,vmax=3000)

[[28, 12], [81, 180], [101, 62], [188, 109], [254, 63]]
Trying to fit peak at (7295.9, 3200.0)
Fitted parameters [6783.896 3199.951 614.914 896.3 0.0 3799171.000000 0.0]
Fitted 1 peaks
Trying to fit peak at (48255.3, 28031.6)
Fitted parameters [48767.256 29311.553 2170.987 3245.9 -4.7 18636.000000 0.0]
Fitted 2 peaks
Trying to fit peak at (25983.6, 15999.8)
Fitted parameters [23935.635 19583.701 2597.687 2230.6 116.3 5789.000000 0.0]
Fitted 3 peaks
Trying to fit peak at (20863.7, 46207.3)
Fitted parameters [17791.729 46207.295 2492.996 3154.3 -7.2 1687.000000 0.0]
Fitted 4 peaks
Trying to fit peak at (65151.0, 16255.8)
Fitted parameters [65407.002 15231.768 72.845 1525.7 0.0 155685.000000 0.0]
Fitted 5 peaks


/home/hektor/scripts/qim/registration/dualHistogramSegmentation.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [155]:
coverage = 0.9
phasediagram = registration.phase_diagram(data,xedges,yedges,opts,coverage=coverage,sigma=4)
nphases=phasediagram.max()+1

In [156]:
#plot the phasediagram
cmap = tools.random_cmap(ncolors=nphases,startmap=plt.cm.Set3)#'Set3' #vill bara ha x färger
Y,X = np.meshgrid(yedges,xedges,indexing='ij') #coordinates in dual histogram
fig=plt.figure()
#m=plt.pcolormesh(X,Y,phasediagram,cmap=cmap)
m=plt.imshow(phasediagram,cmap=cmap,origin='lower')
plt.xlabel('X-ray intensity')
plt.ylabel('Neutron intensity')
cb=fig.colorbar(m,ticks=np.arange(nphases))
m.set_clim(-0.5, nphases - 0.5)


/sw/easybuild/software/IPython/7.9.0-foss-2019b-Python-3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [144]:
nphases

6